##ANN - Pytorch

by Alexandra La Cruz


In [15]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import StratifiedKFold
import numpy as np
import pandas as pd

from sklearn.metrics import classification_report

# Cargar el conjunto de datos Iris
iris = load_iris()
X, y = iris.data, iris.target
# Dividir en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

X = np.concatenate([X_train, X_test], axis=0)
y = np.concatenate([y_train, y_test], axis=0)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Save ONLY the validation features to CSV (no target)
X_val_df = pd.DataFrame(X_val, columns=iris.feature_names)
X_df = pd.DataFrame(X, columns=iris.feature_names)
X_df["target"] = y          # add the label as a new column


# Guardar a CSV sin la columna índice
X_val_df.to_csv("../datasets/tabular/iris_topredict.csv", index=False)
X_df.to_csv("../datasets/tabular/iris_totrain.csv", index=False)

In [7]:

# Escalar los datos / Normalizar los datos
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Convertir a tensores de PyTorch
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)  # Usar torch.long para clasificación multiclase y torch.float32 para clasificación multi-label
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)   # Usar torch.long para clasificación multiclase  y torch.float32 para clasificación multi-label

# Crear DataLoader
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# Instanciar el modelo
input_size = X_train.shape[1]
num_classes = len(np.unique(y))  # Obtener el número de clases únicas

model = nn.Sequential(
    nn.Linear(input_size, 4),
    nn.ReLU(),
    nn.Linear(4, 2),
    nn.ReLU(),
    nn.Linear(2, num_classes))

# Definir la función de pérdida y el optimizador
criterion = nn.CrossEntropyLoss()  # CrossEntropyLoss para clasificación multiclase y nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Bucle de entrenamiento
epochs = 500
for epoch in range(epochs):
    for inputs, labels in train_loader:
        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')

# Evaluación
with torch.no_grad():
    model.eval()
    test_outputs = model(X_test_tensor)
    _, predicted = torch.max(test_outputs, 1)  # Obtener las clases predichas
    accuracy = (predicted == y_test_tensor).float().mean()
    print(classification_report(y_test_tensor, predicted))
#


Epoch [10/500], Loss: 1.1701
Epoch [20/500], Loss: 1.1932
Epoch [30/500], Loss: 1.1417
Epoch [40/500], Loss: 1.0706
Epoch [50/500], Loss: 1.1012
Epoch [60/500], Loss: 1.1499
Epoch [70/500], Loss: 1.0792
Epoch [80/500], Loss: 1.0692
Epoch [90/500], Loss: 1.0999
Epoch [100/500], Loss: 1.0267
Epoch [110/500], Loss: 1.0617
Epoch [120/500], Loss: 0.9818
Epoch [130/500], Loss: 0.9670
Epoch [140/500], Loss: 1.0418
Epoch [150/500], Loss: 0.9320
Epoch [160/500], Loss: 0.9755
Epoch [170/500], Loss: 1.0059
Epoch [180/500], Loss: 0.9704
Epoch [190/500], Loss: 0.8279
Epoch [200/500], Loss: 0.8825
Epoch [210/500], Loss: 0.8470
Epoch [220/500], Loss: 0.9312
Epoch [230/500], Loss: 0.8251
Epoch [240/500], Loss: 0.7740
Epoch [250/500], Loss: 0.8993
Epoch [260/500], Loss: 0.7619
Epoch [270/500], Loss: 0.8676
Epoch [280/500], Loss: 0.7758
Epoch [290/500], Loss: 0.7284
Epoch [300/500], Loss: 0.7323
Epoch [310/500], Loss: 0.6947
Epoch [320/500], Loss: 0.6703
Epoch [330/500], Loss: 0.7269
Epoch [340/500], Lo

In [5]:
from sklearn.metrics import classification_report

print(classification_report(y_test_tensor, predicted))


              precision    recall  f1-score   support

           0       1.00      1.00      1.00        10
           1       1.00      0.89      0.94         9
           2       0.92      1.00      0.96        11

    accuracy                           0.97        30
   macro avg       0.97      0.96      0.97        30
weighted avg       0.97      0.97      0.97        30



In [ ]:
X

array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2],
       [4.6, 3.1, 1.5, 0.2],
       [5. , 3.6, 1.4, 0.2],
       [5.4, 3.9, 1.7, 0.4],
       [4.6, 3.4, 1.4, 0.3],
       [5. , 3.4, 1.5, 0.2],
       [4.4, 2.9, 1.4, 0.2],
       [4.9, 3.1, 1.5, 0.1],
       [5.4, 3.7, 1.5, 0.2],
       [4.8, 3.4, 1.6, 0.2],
       [4.8, 3. , 1.4, 0.1],
       [4.3, 3. , 1.1, 0.1],
       [5.8, 4. , 1.2, 0.2],
       [5.7, 4.4, 1.5, 0.4],
       [5.4, 3.9, 1.3, 0.4],
       [5.1, 3.5, 1.4, 0.3],
       [5.7, 3.8, 1.7, 0.3],
       [5.1, 3.8, 1.5, 0.3],
       [5.4, 3.4, 1.7, 0.2],
       [5.1, 3.7, 1.5, 0.4],
       [4.6, 3.6, 1. , 0.2],
       [5.1, 3.3, 1.7, 0.5],
       [4.8, 3.4, 1.9, 0.2],
       [5. , 3. , 1.6, 0.2],
       [5. , 3.4, 1.6, 0.4],
       [5.2, 3.5, 1.5, 0.2],
       [5.2, 3.4, 1.4, 0.2],
       [4.7, 3.2, 1.6, 0.2],
       [4.8, 3.1, 1.6, 0.2],
       [5.4, 3.4, 1.5, 0.4],
       [5.2, 4.1, 1.5, 0.1],
       [5.5, 4.2, 1.4, 0.2],
       [4.9, 3

In [9]:

# Función para crear el modelo
def create_model(input_size, hidden_sizes, num_classes):
    layers = []
    layers.append(nn.Linear(input_size, hidden_sizes[0]))
    layers.append(nn.ReLU())
    for i in range(len(hidden_sizes) - 1):
        layers.append(nn.Linear(hidden_sizes[i], hidden_sizes[i+1]))
        layers.append(nn.ReLU())
    layers.append(nn.Linear(hidden_sizes[-1], num_classes))
    return nn.Sequential(*layers)

# Función para entrenar y evaluar el modelo
def train_and_evaluate(model, train_loader, val_loader, epochs=100, lr=0.001):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    for epoch in range(epochs):
        model.train()
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return correct / total

In [10]:
# Función de validación cruzada
def cross_validate(X_tensor, y_tensor, hidden_sizes_list, epochs=100, lr=0.001, k=5):
    skf = StratifiedKFold(n_splits=k, shuffle=True, random_state=42)
    results = {}

    for hidden_sizes in hidden_sizes_list:
        accuracies = []
        for train_index, val_index in skf.split(X_tensor, y_tensor):
            X_train, X_val = X_tensor[train_index], X_tensor[val_index]
            y_train, y_val = y_tensor[train_index], y_tensor[val_index]

            train_dataset = TensorDataset(X_train, y_train)
            val_dataset = TensorDataset(X_val, y_val)
            train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
            val_loader = DataLoader(val_dataset, batch_size=16)

            model = create_model(X_tensor.shape[1], hidden_sizes, len(np.unique(y)))
            accuracy = train_and_evaluate(model, train_loader, val_loader, epochs, lr)
            accuracies.append(accuracy)
        results[str(hidden_sizes)] = np.mean(accuracies)
    return results

In [11]:
# Definir las arquitecturas a probar
hidden_sizes_list = [
    [8,4],  # 2 capa, 8 y 4 neuronas
    [20, 10], # 2 capas, 10 y 8 neuronas
    [30, 20, 10], # 3 capas, 30,20 y 10 neuronas
    [64,32], # 2 capas, 64 y 32 neuronas
]

# Ejecutar la validación cruzada
results = cross_validate(X_train_tensor, y_train_tensor, hidden_sizes_list, epochs=200, lr=0.001, k=10)

# Imprimir los resultados
for architecture, accuracy in results.items():
    print(f"Arquitectura: {architecture}, Precisión promedio: {accuracy:.4f}")

# Encontrar la mejor arquitectura
best_architecture = max(results, key=results.get)
best_accuracy = results[best_architecture]
print(f"\nMejor arquitectura: {best_architecture}, Mejor precisión promedio: {best_accuracy:.4f}")



Arquitectura: [8, 4], Precisión promedio: 0.9467
Arquitectura: [20, 10], Precisión promedio: 0.9789
Arquitectura: [30, 20, 10], Precisión promedio: 0.9678
Arquitectura: [64, 32], Precisión promedio: 0.9567

Mejor arquitectura: [20, 10], Mejor precisión promedio: 0.9789


In [12]:
eval(best_architecture)

[20, 10]

In [13]:
model = create_model(X_test.shape[1], eval(best_architecture), len(np.unique(y)))

# Evaluación
with torch.no_grad():
    model.eval()
    test_outputs = model(X_test_tensor)
    _, predicted = torch.max(test_outputs, 1)  # Obtener las clases predichas
    accuracy = (predicted == y_test_tensor).float().mean()
    print(f'Test Accuracy: {accuracy.item():.4f}')

Test Accuracy: 0.4667
